# Face Attendence Sysytem

In [ ]:
User Input:
The program prompts the user to input their name and student ID. This information is required to log attendance.
    
Face Detection:
The program uses OpenCV's Haar Cascade classifier to detect faces from the webcam feed. It converts each captured frame to grayscale and 
applies the classifier to identify faces.

Unique Face Identification:
Once a face is detected, the program creates a unique Face ID by converting the cropped face image into a hash using the SHA-256 algorithm. 
This ensures that the face can be consistently identified in the future.
    
Attendance Logging:
The program checks if the Face ID already exists in the attendance.csv file. If the Face ID is not found, it logs the student’s name, ID, 
                                                    Face ID, and the current timestamp in the CSV file.
Prevention of Duplicate Entries:

To ensure a person’s attendance is only recorded once, the program tracks Face IDs using a set. Once a face is detected and logged, its Face ID is stored, and any future detections of the same face are ignored.
Automatic Webcam Closure:

After logging attendance, the program automatically releases the webcam and closes the OpenCV window to finish the process.
This approach ensures accurate, unique, and one-time logging of attendance based on facial recognition.

In [16]:
import cv2
import csv
import os
import hashlib
import numpy as np
from datetime import datetime

# Function to check if Face ID already exists in the CSV
def check_existing_entry(face_id, file_path="attendance.csv"):
    if os.path.exists(file_path):
        with open(file_path, mode='r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[2] == face_id:  # The Face ID is in the third column (index 2)
                    return True  # Entry exists
    return False  # Entry doesn't exist

# Function to mark attendance in the CSV file
def mark_attendance(name, student_id, face_id, file_path="attendance.csv"):
    if not check_existing_entry(face_id, file_path):
        file_exists = os.path.exists(file_path)
        
        with open(file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            
            if not file_exists:
                writer.writerow(["ID", "Name", "Face ID", "Time"])
            
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow([student_id, name, face_id, current_time])
            
            # Display the attendance entry in the console
            print(f"Attendance for {name} logged successfully.")
            print(f"ID: {student_id}, Face ID: {face_id}, Time: {current_time}")
    else:
        print(f"Entry for {name} already exists.")

# Function to generate a unique face ID by hashing the face image
def generate_face_id(face_image):
    # Convert the image to grayscale
    gray_face = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    # Resize the face image to a fixed size for consistency
    resized_face = cv2.resize(gray_face, (100, 100))
    # Compute the hash of the resized image
    face_id = hashlib.sha256(resized_face.tobytes()).hexdigest()
    return face_id

# Function for face detection and recording attendance
def detect_face_and_record(name, student_id):
    # Load OpenCV's pre-trained Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Initialize the webcam feed
    cap = cv2.VideoCapture(0)
    
    # Flag to track if face is already detected and logged
    logged_faces = set()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # If faces are detected
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Crop the face region from the frame
                face_image = frame[y:y+h, x:x+w]
                
                # Generate a unique Face ID from the cropped face
                face_id = generate_face_id(face_image)
                
                # If the face hasn't been logged yet, mark attendance and add the face ID to the set
                if face_id not in logged_faces:
                    mark_attendance(name, student_id, face_id)
                    logged_faces.add(face_id)  # Mark face as logged
                    print(f"Attendance for {name} recorded.")
                    # Automatically close the webcam after logging attendance
                    cap.release()
                    cv2.destroyAllWindows()
                    return  # Exit after logging attendance
        
        # Display the frame with face detection
        cv2.imshow("Face Detection", frame)
        
    # Release the webcam and close all windows after logging the attendance
    cap.release()
    cv2.destroyAllWindows()

def main():
    # Collect user input for name and ID
    name = input("Enter your name: ")
    student_id = input("Enter your student ID: ")

    # Detect face and log attendance
    detect_face_and_record(name, student_id)

if __name__ == "__main__":
    main()


Enter your name:  Pranay Dhore
Enter your student ID:  22047


Attendance for Pranay Dhore logged successfully.
ID: 22047, Face ID: 3ec2c1f0e5cf4688b208d439c75a11e292ddba5510aadb213f1f584f3c7ce311, Time: 2024-12-22 22:20:12
Attendance for Pranay Dhore recorded.
